# OpenWPM - Análisis estilo Lightbeam

Este Notebook:
1. Carga los datos de una ejecución previa de OpenWPM (archivo SQLite).
2. Identifica las peticiones a "terceros" (third-party) al comparar el dominio de la página visitada vs. el dominio del recurso.
3. Contesta a preguntas habituales tipo Lightbeam:
   - Número total de terceros
   - Página con más terceros
   - Dos páginas con más terceros en común
   - Tercero más frecuente


In [3]:
import sqlite3
import pandas as pd
from urllib.parse import urlparse
from itertools import combinations

DB_PATH = "/workspaces/OpenWPM/datadir/crawl-data.sqlite"

# 1. Conectamos con la base de datos SQLite que generó OpenWPM
conn = sqlite3.connect(DB_PATH)

# 2. Cargamos la tabla http_requests, donde se registran las peticiones
df_requests = pd.read_sql_query("SELECT * FROM http_requests", conn)
conn.close()

print("Columnas disponibles en http_requests:\n", df_requests.columns.tolist())
print("\nEjemplo de filas:")
display(df_requests.head(5))

Columnas disponibles en http_requests:
 ['id', 'incognito', 'browser_id', 'visit_id', 'extension_session_uuid', 'event_ordinal', 'window_id', 'tab_id', 'frame_id', 'url', 'top_level_url', 'parent_frame_id', 'frame_ancestors', 'method', 'referrer', 'headers', 'request_id', 'is_XHR', 'is_third_party_channel', 'is_third_party_to_top_window', 'triggering_origin', 'loading_origin', 'loading_href', 'req_call_stack', 'resource_type', 'post_body', 'post_body_raw', 'time_stamp']

Ejemplo de filas:


,id,incognito,browser_id,visit_id,extension_session_uuid,event_ordinal,window_id,tab_id,frame_id,url,...,is_third_party_channel,is_third_party_to_top_window,triggering_origin,loading_origin,loading_href,req_call_stack,resource_type,post_body,post_body_raw,time_stamp
0,1,0,496321143,3162082775655329,a262be8d-1a58-4033-bb0b-7c0232a3437c,1,1.0,1,0,http://www.example.com/,...,None,None,undefined,undefined,undefined,None,main_frame,None,None,2025-02-16T18:42:20.421Z
1,2,0,1869619805,3177703552783071,eb93d226-e36f-44f2-b901-0002e073e1f9,1,1.0,1,0,http://www.princeton.edu/,...,None,None,undefined,undefined,undefined,None,main_frame,None,None,2025-02-16T18:42:20.436Z
2,3,0,1869619805,3177703552783071,eb93d226-e36f-44f2-b901-0002e073e1f9,3,1.0,1,0,https://www.princeton.edu/,...,None,None,undefined,undefined,undefined,None,main_frame,None,None,2025-02-16T18:42:20.520Z
3,4,0,496321143,3162082775655329,a262be8d-1a58-4033-bb0b-7c0232a3437c,5,1.0,1,0,http://www.example.com/favicon.ico,...,None,None,http://www.example.com,http://www.example.com,http://www.example.com/,None,image,None,None,2025-02-16T18:42:20.789Z
4,5,0,1869619805,3177703552783071,eb93d226-e36f-44f2-b901-0002e073e1f9,7,1.0,1,0,https://www.princeton.edu/sites/default/files/...,...,None,None,https://www.princeton.edu,https://www.princeton.edu,https://www.princeton.edu/,None,script,None,None,2025-02-16T18:42:20.743Z


### 3. Normalizar dominios
Vamos a extraer el dominio de la página visitada (columna `top_level_url`) y el dominio del recurso (`url`) para identificar si es un tercero.

In [4]:
def get_domain_from_url(u: str) -> str:
    if not u:
        return ""
    parsed = urlparse(u)
    return parsed.netloc.lower()

df_requests["page_domain"] = df_requests["top_level_url"].apply(get_domain_from_url)
df_requests["req_domain"]  = df_requests["url"].apply(get_domain_from_url)

print(df_requests[["top_level_url", "page_domain", "url", "req_domain"]].head())

                top_level_url        page_domain  \
0     http://www.example.com/    www.example.com   
1   http://www.princeton.edu/  www.princeton.edu   
2  https://www.princeton.edu/  www.princeton.edu   
3     http://www.example.com/    www.example.com   
4  https://www.princeton.edu/  www.princeton.edu   

                                                 url         req_domain  
0                            http://www.example.com/    www.example.com  
1                          http://www.princeton.edu/  www.princeton.edu  
2                         https://www.princeton.edu/  www.princeton.edu  
3                 http://www.example.com/favicon.ico    www.example.com  
4  https://www.princeton.edu/sites/default/files/...  www.princeton.edu  


### 4. Identificar peticiones a terceros
Definimos que una petición es de "terceros" si el dominio del recurso (`req_domain`) difiere del dominio de la página (`page_domain`).

In [5]:
# Creamos una columna booleana is_thirdparty
df_requests["is_thirdparty"] = df_requests.apply(
    lambda row: row["req_domain"] != row["page_domain"], axis=1
)
df_third = df_requests[df_requests["is_thirdparty"]].copy()
print(f"Total de peticiones: {len(df_requests)}")
print(f"Total de peticiones a terceros: {len(df_third)}")

Total de peticiones: 872
Total de peticiones a terceros: 380


### 5. Calcular el número total de terceros
Obtenemos los dominios de terceros únicos:

In [6]:
all_third_parties = df_third["req_domain"].unique()
num_third_parties = len(all_third_parties)

print("Número total de dominios terceros únicos:", num_third_parties)

Número total de dominios terceros únicos: 43


### 6. Página con más terceros
Agrupamos por la página visitada y contamos cuántos terceros *distintos* aparecen en cada una.

In [7]:
# Agrupamos en sets para no contar repetidos
grouped_sets = df_third.groupby("page_domain")["req_domain"].agg(set)
# El resultado es un Series: page_domain -> set({varios_terceros})

df_counts = grouped_sets.apply(len).reset_index(name="count_third")
page_with_max = df_counts.loc[df_counts["count_third"].idxmax()]
print("Página con más terceros:", page_with_max["page_domain"], 
      "\nNúmero de terceros:", page_with_max["count_third"])

Página con más terceros: www.elcorteingles.es 
Número de terceros: 17


### 7. Dos páginas con más terceros en común
Para cada par de sitios, calculamos la intersección de sus sets de dominios terceros:

In [8]:
page_to_thirdset = grouped_sets.to_dict()  # dict: { 'page_domain' -> set({terceros...}) }

all_pages = list(page_to_thirdset.keys())
max_shared = -1
best_pair = (None, None)

for p1, p2 in combinations(all_pages, 2):
    shared = page_to_thirdset[p1].intersection(page_to_thirdset[p2])
    scount = len(shared)
    if scount > max_shared:
        max_shared = scount
        best_pair = (p1, p2)

print("Las dos páginas con más terceros en común son:", best_pair)
print("Número de terceros compartidos:", max_shared)

Las dos páginas con más terceros en común son: ('www.elcorteingles.es', 'www.uoc.edu')
Número de terceros compartidos: 5


### 8. Tercero más frecuente
Buscamos en cuántos **sitios** aparece cada dominio tercero.

In [9]:
# Transformamos page -> set(terceros) a una lista [(page, dom), ...]
pairs = []
for page, sdoms in page_to_thirdset.items():
    for d in sdoms:
        pairs.append((page, d))

df_pairs = pd.DataFrame(pairs, columns=["page_domain", "third_domain"])
# Contamos en cuántas páginas aparece cada third_domain
counts = df_pairs["third_domain"].value_counts()

most_common_third = counts.index[0]
freq = counts.iloc[0]
print("Tercero más frecuente:", most_common_third)
print("Aparece en", freq, "páginas distintas de", len(all_pages), "totales.")

Tercero más frecuente: www.googletagmanager.com
Aparece en 5 páginas distintas de 7 totales.


## Conclusiones
Con este Notebook hemos:
- Cargado los datos generados por OpenWPM.
- Identificado peticiones de terceros comparando dominios.
- Obtenido:
  1. Número total de terceros
  2. Página con más terceros
  3. Dos páginas con más terceros en común
  4. Tercero que aparece en más páginas

Puedes ajustar la lógica de "tercero" (e.g., ignorar subdominios), añadir más análisis, o usar cualquier otra tabla (`http_responses`, `cookies`, etc.) según necesites.